# Scrape website 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' for all Toronto neighborhoods

import libraries for data analysis (np and pd) and accessing website data (requests and BeautifulSoup)

In [3]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

access website and download all text

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
website_url = requests.get(url).text
soup = BeautifulSoup(website_url,'lxml')


Read in main table data to variable My_table with column headers PostCode, Borough, and Neighborhood.  Read by table row and separate by text td.  Drop missing values.  Strip out strings \n

In [5]:
My_table = soup.find('table',{'class':'wikitable sortable'})
rows  = My_table.find_all('tr')
TN=[]
for tr in rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    #print(row)
    TN.append(row)
df=pd.DataFrame(TN, columns=['Postal Code', 'Borough', 'Neighborhood'])
df = df.replace(to_replace='None', value=np.nan).dropna()
df['Neighborhood']=df['Neighborhood'].str.strip('\n')


Drop row if Borough Not assigined.  
If Neighborhood set to Not assigned replace with borough name.  
Group by postcode and borough.  
Assign one row for each postal code, in neighborhood column place all neighborhoods separated by comma.
Place Result into variable TN

In [6]:

df['Borough'] = df['Borough'].replace(to_replace="Not assigned", value=np.nan)
df=df.dropna()

df['Neighborhood'] = df['Neighborhood'].replace(to_replace="Not assigned", value=np.nan)
values=df['Borough']
df['Neighborhood'] = values.where(df['Neighborhood'].isnull(), other=df['Neighborhood'])

TN=df.groupby(['Postal Code', 'Borough'], sort=False)['Neighborhood'].apply(','.join).reset_index()

TN.columns=('Postal Code', 'Borough', 'Neighborhood')


Find number of rows of data with .shape

In [7]:
TN.shape

(103, 3)

Find longitude and latitude of each neighborhood and append to TN variable, create new variable TNLL to hold all data

In [15]:
Coords=pd.read_csv('http://cocl.us/Geospatial_data')

LatLon=pd.DataFrame()
LatLon=Coords

TN.columns=('Postal Code', 'Latitude', 'Longitude')

LatLon.head()

TNLL=pd.merge(TN, LatLon, on="Postal Code")

TNLL.columns=('Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude')

TNLL.head(10)




,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
